In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

weather_data = pd.read_csv('../input/weather-dataset-rattle-package/weatherAUS.csv')

weather_data.head()

# **CÁC ĐỊNH NGHĨA VỀ KHOẢNG CÁCH**

> 1. Chuẩn 2 ( L2 Norm) Khoảng cách Euclide

√(x₁₂-x₂₂)²+(y₁₂-y₂₂)² =  ||x₁-x₂||₂

We can also rewrite this as 

||x₁-x₂||₂ =   (∑(x₁ᵢ-x₂ᵢ)²)¹/² where i=1 to n

The right side of this equation is called L2 Norm

> 2. Chuẩn 1 (L1 Norm) khoảng cách mahattan

distance =(x₁₂-x₂₂)  which can also be rewritten as

∑ abs(x₁ᵢ-x₂ᵢ) where i=1 to n

This is called L1 norm

3. > Chuẩn p (Lp Norm) khoảng cách Minkowski

||x₁-x₂||ₚ =(∑(x₁ᵢ-x₂ᵢ)ᵖ)¹/ᵖ

This is also called the Lp norm

If p=1, then it is Manhattan distance

If p=2, then it is Euclidean distance

# **CHỌN K NHƯ THẾ NÀO ?**

Với bài toán phân loại nhị phân, chúng ta sẽ chọn K lẻ để đảm bảo nhãn sẽ được phân loại

Trong tất cả các trường hợp người ta chưa có thống kê nào cho thấy giá trị K nào là tốt nhất.

Vậy Chúng ta sẽ cho K chạy trong 1 tập hữu hạn và chọn giá trị K làm cho giá trị hàm error là nhỏ nhất. 

# **XỬ LÝ THẾ NÀO KHI 2 NHÃN CÓ CÙNG SỐ LƯỢNG HÀNG XÓM VOTE ?**

- Random 
- Tăng hoặc giảm K cho đến khi có sự chênh lệch lượng vote giữa 2 nhãn ( giảm accuracy)
- Đánh trọng số cho mỗi ứng viên và lấy giá trị trọng số này để so sánh giữa 2 nhãn nếu có số lượng vote = nhau

# **TẠI SAO LÀM VIỆC VỚI KNN PHẢI NORMALIZE DATA ?**

- Vì các feature sẽ có những đặc trưng khác nhau và không thuộc cùng 1 dài giá trị để tính khoảng cách

- Ví dụ : Người 1 : Age = 25 , income = 80.000 ; Người 2 : Age = 30 , income = 100.000

Khoảng cách d = ((Age1-Ag2)^2 + (income1-income2)^2)^(1/2) = 20000.000625 

Vậy ảnh hưởng của Age không quá lớn đối với hàm khoảng cách nhưng trên thực tế Age có ảnh hưởng khá lớn tới thu nhập cũng như các feature khác thì model sẽ rất tệ.

- Chúng ta cần tập dữ liệu có phân phối xác suất vậy thì cần phải nomalize

# **CÁC KĨ THUẬT NORMALIZE DATA ?**

> 1. Scalling 

Thay đổi khoảng giá trị của Data

minmax_scaling trong mlxtend.preprocessing

> 2. Normalizing

Thay đổi phân phối xác suất của các điểm dữ liệu

boxcox trong Stats thư viện scipy

# **IMPLEMENT THUẬT TOÁN KNN**

In [ ]:
weather_data.head()

In [ ]:
weather_data['RainTomorrow'].value_counts()

In [ ]:
weather_data.isnull().sum().sort_values(ascending = False)

****Xóa các cột có số lượng NULL > 28%****

In [ ]:
max_nan = len(weather_data)*0.28
weather_data = weather_data.loc[:,(weather_data.isnull().sum(axis=0) <= max_nan) ]
weather_data

In [ ]:
weather_data.isnull().sum().sort_values(ascending = False)

**Điền các giá trị còn thiếu bởi next value**

In [ ]:
weather_data.dropna(subset = ["RainTomorrow"], inplace = True)
weather_data.fillna(method='bfill' , inplace=True)

In [ ]:
weather_data.isnull().sum()

In [ ]:
# loại bỏ 1 số cột không có ý nghĩa
weather_data.drop(columns= ['Date','Location'] , inplace=True)

In [ ]:
weather_data['RainToday'] = weather_data['RainToday'].astype('category')
weather_data['RainTomorrow'] = weather_data['RainTomorrow'].astype('category')


In [ ]:
# Chuyển về numeric
weather_data['RainToday'] = weather_data['RainToday'].cat.codes
weather_data['RainTomorrow'] = weather_data['RainTomorrow'].cat.codes


In [ ]:
col_is_numeric = [col for col in weather_data.columns if weather_data[col].dtype in ['int64','float64','int8']]
weather_data = weather_data[col_is_numeric]

In [ ]:
Full_Data_X = weather_data
X = Full_Data_X.drop(columns= 'RainTomorrow' , axis = 1)
# X = Full_Data_X.drop(columns= 'RainToday' , axis = 1)
y = Full_Data_X['RainTomorrow']

In [ ]:
# X=pd.get_dummies(X,columns=['WindDir9am','WindDir3pm','WindGustDir'],drop_first=True)

In [ ]:
#Scale data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X), index=X.index, columns=X.columns)

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.25 ,random_state=0,stratify=y)

In [ ]:
# Chọn K
K = 101

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

classifier = KNeighborsClassifier(n_neighbors = 13 , p = 2 , weights = 'distance')
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)
score = accuracy_score(y_test,y_pred)
print("Predicted labels: ", y_pred[1:20])
print("Ground truth    : ", np.array(y_test[1:20]))

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy of KNN: %.2f %%" %(100*accuracy_score(y_test, y_pred)))

In [ ]:
import math

In [ ]:
def knn(row):
    x = np.array(row)
    candidate = []
    for i in range(len(X_train)):
        y = np.array(X_train.iloc[i])
        score = (y-x).dot(y-x)
        candidate.append([math.sqrt(score),y_train.iloc[i]])
    candidate.sort()
    candidate = candidate[0:K]
    vote = 0
    for i in candidate:
        if i[1]==1:
            vote+=1
    if vote>K//2:
        return 1
    else:
        return 0

In [ ]:
# predict = []
# for i in range(len(X_test)):
#     predict.append(knn(X_test.iloc[i]))
# print("Accuracy of KNN: %.2f %%" %(100*accuracy_score(y_test, np.array(predict))))

In [ ]:
import time
second = time.time()
x = np.array(X_train)
y = np.array(X_test.iloc[0])
candidate = np.zeros(len(x))
for i in range(x.shape[0]):
    score = (x[i]-y).dot(x[i]-y)
    candidate[i] = math.sqrt(score)
candidate.sort(kind = 'quicksort')
candidate = candidate[0:K]

print(candidate)
print(time.time() - second)

# **các kĩ thuật tối ưu khi search các hàng xóm gần nhất ?**

> 1. KD-Tree

Thuật toán tìm kiếm dựa trên binary search tree

mỗi khi đi xuống 1 nhánh sử dụng 1 chiều dữ liệu luân phiên

>2. KNN with LSH ( locality sensitive hashing)

Chia tập dữ liệu ra thành các region. Nếu test_point thuộc region nào thì chỉ xét các neighbor thuộc region đó.

>3. KNN with Inverted List